# Anomaly Detection Analysis

In [1]:
import pandas as pd
from src.constants import FILEPATH_USECASE, FILEPATH_PREDICTIONS
import os

In [2]:
df = pd.read_csv(os.path.join(FILEPATH_USECASE, 'results_anomaly_bpi_2018_scores.csv'))

In [3]:
df.sort_values('severity')[::-1]

,violation,severity,type,event1,event2
30,refuse occurred together with approve,1.000000,XOR,refuse,approve
28,decide occurred without check,0.750000,CO_OCC,decide,check
29,remove document occurred before insert document,0.750000,ORDER,remove document,insert document
27,decide occurred before plan,0.714286,ORDER,decide,plan
26,performed occurred before plan,0.625000,ORDER,performed,plan
25,decide occurred without plan,0.625000,CO_OCC,decide,plan
24,approve occurred before begin editing,0.500000,ORDER,approve,begin editing
23,performed offline occurred before plan,0.375000,ORDER,performed offline,plan
20,performed occurred without plan,0.333333,CO_OCC,performed,plan
22,cancel offline occurred before begin editing,0.333333,ORDER,cancel offline,begin editing


In [4]:
df_pred = pd.read_csv(os.path.join(FILEPATH_PREDICTIONS, 'svc.csv'))
df_pred.index = df_pred['concept:name']
df_pred

,concept:name,annotation,pred
concept:name,,,
A_Accepted,A_Accepted,decide,decide
A_Cancelled,A_Cancelled,decide,decide
A_Complete,A_Complete,transform,transform
A_Concept,A_Concept,manage,transform
A_Create Application,A_Create Application,create,create
...,...,...,...
Release B,Release B,transform,transform
Release C,Release C,transform,transform
Release D,Release D,transform,transform


In [5]:
df['role1'] = list(df_pred.loc[df['event1']]['annotation'])
df['role2'] = list(df_pred.loc[df['event2']]['annotation'])

In [6]:
pd.concat([df['role1'], df['role2']]).value_counts()

transform    25
decide       15
manage        8
assess        7
preserve      2
create        2
move          1
destroy       1
combine       1
dtype: int64

In [7]:
df[['role1', 'role2']].value_counts()

role1      role2    
transform  manage       5
           transform    5
decide     assess       3
           transform    3
assess     transform    2
decide     manage       2
transform  decide       2
assess     decide       1
create     transform    1
decide     decide       1
destroy    combine      1
manage     transform    1
move       decide       1
preserve   assess       1
           decide       1
transform  create       1
dtype: int64

In [8]:
df.groupby(['type', 'role1', 'role2']).count()

violation  severity  event1  event2
type   role1     role2                                         
CO_OCC decide    assess             1         1       1       1
                 manage             1         1       1       1
       manage    transform          1         1       1       1
       transform manage             2         2       2       2
                 transform          1         1       1       1
ORDER  assess    decide             1         1       1       1
       decide    manage             1         1       1       1
                 transform          2         2       2       2
       destroy   combine            1         1       1       1
       transform decide             1         1       1       1
                 manage             3         3       3       3
                 transform          4         4       4       4
XOR    assess    transform          2         2       2       2
       create    transform          1         1       1       1
       decide    assess             2         2       2       2
                 decide             1         1       1       1
                 transform          1         1       1       1
       move      decide             1         1       1       1
       preserve  assess             1         1       1       1
                 decide             1         1       1       1
       transform create             1         1       1       1
                 decide             1         1       1       1